# Introduction To Bayesian Optimization 

Hitarth Choubisa


<h2> <center> Outline  </center> </h2>

* Motivation: Why go beyond Supervised and Unsupervised Learning?

* Drawbacks of Supervised Learning algorithms

* Gaussian Process Regression (GPR)

* Bayesian Optimization (BO) using GPR

* Python for BO

<h2> <center> Bayesian Optimization  </center> </h2>

* Can ML tell us **the next best experiment?**
* Can ML tell us where we have uncertainty in our experimental space?
* Bayesian Optimization allows us to answer both questions!
* Algorithms we have covered thus far lack this ability

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

def plot_gp(mu, cov, X, X_train=None, Y_train=None, samples=[]):
    X = X.ravel()
    mu = mu.ravel()
    uncertainty = 1.96 * np.sqrt(np.diag(cov))
    
    plt.fill_between(X, mu + uncertainty, mu - uncertainty, alpha=0.1)
    plt.plot(X, mu, label='Mean')
    for i, sample in enumerate(samples):
        plt.plot(X, sample, lw=1, ls='--', label=f'Sample {i+1}')
    if X_train is not None:
        plt.plot(X_train, Y_train, 'rx')
    plt.legend()

def plot_gp_2D(gx, gy, mu, X_train, Y_train, title, i):
    ax = plt.gcf().add_subplot(1, 2, i, projection='3d')
    ax.plot_surface(gx, gy, mu.reshape(gx.shape), cmap=cm.coolwarm, linewidth=0, alpha=0.2, antialiased=False)
    ax.scatter(X_train[:,0], X_train[:,1], Y_train, c=Y_train, cmap=cm.coolwarm)
    ax.set_title(title)

<h2> <center> What does uncertainty mean?</center> </h2>

![](pics_for_a3md/missing_data.png)

<h2> <center> Supervised Learning  </center> </h2>

* Supervised Machine Learning: Given input, predict (Continuous or Discrete) output 

![](pics_for_a3md/sup_learning.png)

* Logisitic Regression, Support Vector Machines, Random Forests etc..
* Problems: These algorithms to not work well with:
    * A large input space
    * Small datasets
* The solution: Gaussian Process Regression

<h2> <center> Gaussian Process Regression  </center> </h2>

* There are two ways to approach ML: 
    * Probablistic Approach: (like Naive Bayes)
    * Non-probablistic approach: (Logistic Regression, Perceptron, SVM, etc.)


* Naive Bayes: Every dimension of input(X) is a Random Variable; y is the target
    $$P(X|y)=\prod_{n=1}^d P(x_n|y)$$
    
$X=[x_1,x_2,...,x_d]$

* and we try to find the class, y, that maximixes the function
$$P(y|X) = P(x|y)P(y)$$
    

## Gaussian Processes

* What if we model every datapoint as Random Variable?
    $$P(y|X) = P(y_1,...,y_N|[x_1,x_2,...,x_N];[w_1,...,w_k])$$
    $X: \text{ \{Collection of all the input pairs\}}$, 
     N: number of datapoints, $w_i$: parameters of the ML model
  

## Probabilities: Joint and Conditional
* A random variable X can be defined given its probability distribution $p(X)$
* Given two random variables X and Y, their joint probability is $p(X,Y)$ and it satisfies $\int_{R} \int_{R}p(x,y)dx dy = 1$ and $p(X,Y) \geq 0$
* Also, given a joint distribution, the conditional probability distribution can be defined as:
$$p(X|Y) = \frac{p(X,Y)}{p(Y)}$$
* We can recover the distribution for individual random variables from the joint distribution as, $$p(X) = \int_{R}p(x,y)dy$$

## Example of Coin Toss

* Toss two coins together
* Set of all the outcomes is: $\{HH, TT, HT, TH\}$
* Let's calculate different probabilities in this case
* $P(C_1=H,C_2=H) = P(HH) = P(C_1=H) \cdot P(C_2=H) = 1/2 * 1/2 = 0.25 $

* $P(HH) = P(HT) = P(TH) = P(TT) = \frac{1}{4}$

* $C_1$: Coin 1, $C_2$: Coin 2 


## Coin Toss example

* To recover individual distribution:
<br>
$$p(X) = \int_{R}p(x,y)dy$$
$$P(C_1=H) = P(C_1=H,C_2=H)+P(C_1=H, C_2=T) = 1/2$$

* To get the conditional distribution:
<br>
$$p(X|Y) = \frac{p(X,Y)}{p(Y)}$$
$$P(C_2=T|C_1=H) = \frac{P(C_1=H,C_2=T)}{P(C_1=H)} = \frac{P(HT)}{0.5} = \frac{0.25}{0.5} = 0.5$$

## Gaussian Process (GP)
* **Definition**: GP is a collection of random variables (RV) such that the joint distribution of every finite subset of RVs is multivariate Gaussian
* The multivariate Gaussian distribution is defined by a mean vector $\mu$ and a covariance matrix $\Sigma$. We denote this using a simple notation: $N(\mu, \Sigma)$. We call $\Sigma$ co-variance matrix.



In [ ]:
# Normal distribution
import numpy as np

x = np.linspace(-3.,3.,100)
mu = 1
sigma = 0.7
y = 1/np.sqrt(2*sigma**2)*np.exp(-0.5*(x-mu)**2/sigma**2)
plt.plot(x,y)
plt.ylim(0,8)
plt.vlines(mu,0,max(y),'r','dashed')
plt.ylabel('Probability density function')


![](pics_for_a3md\g_rvs.png)

* For a multidimensional Gaussian distribution, the entries of $\Sigma$ dictates how fast the decay happens in different directions. 

## Latent/Objective Function:
* Goal of Gaussian processes: Learn the underlying multivariate distribution from training data.

* Let *f* be the function we want to optimize: we call it **latent function** or **objective function**: $f(x_n) \rightarrow y_n$
* *f* can be anything: experimental mapping, radioactivity decay, expensive-to-evaluate mathematical function




## Gaussian Process

* A Gaussian process is a random process where the joint distribution of a finite number of these variables $p(f(x1),…,f(xN))$ is itself Gaussian: $$p(f|X)=N(\mu,\Sigma)$$
<br>
where $f=(f(x_1),f(x_2),...,f(x_N)), \mu=(m(x_1),m(x_2),..,m(x_N))$ and $\Sigma_{ij}=\kappa(x_i,x_j)$. **m** is the mean function and $\kappa$ is called a kernel function which is used for capturing how close two input pairs are.


* As we observe new outputs, we improve upon this: prior to posterior

## Kernels
* Recall that in order to set up our distribution, we need to define $\mu$ and $\Sigma$.
* In Gaussian processes it is often assumed that $\mu = 0$ 
* The key step of Gaussian processes is setting up the covariance matrix $\Sigma$:

    * We do this by evaluating the kernel $\kappa$, pairwise on all the points. 


* The kernel receives two points $x,x'\in \mathbb{R}^n$ as an input and returns a similarity measure between those points in the form of a scalar.
* Might have seen this in context of SVMs
* kernels allow similarity measures beyond the standard euclidean distance (L2-distance)
* A few common kernels are shown below:

![](pics_for_a3md/kernels.png)

In [ ]:
import numpy as np

def kernel(X1, X2, l=1.0, sigma_f=1.0):
    '''
    Isotropic squared exponential kernel. Computes 
    a covariance matrix from points in X1 and X2.
    
    Generating covariance matrix for training data,
    we just give the same input twice to X1 and X2
        
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        Covariance matrix (m x n).
    '''
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)


In [ ]:
x1 = np.linspace(-3,3,100).reshape(-1,1)
x2 = x1.copy()

outs = kernel(x1,x2)
plt.imshow(outs)
plt.colorbar()
#plt.plot()

## The Prior distribution p(f|X)
* From the definition of GPR,  $$p(f|X)=N(\mu,\Sigma)$$
<br>
where $f=(f(x_1),f(x_2),...,f(x_N)), \mu=(m(x_1),m(x_2),..,m(x_N))$ and $\Sigma_{ij}=\kappa(x_i,x_j)$. 
* **m** is the mean function and $\kappa$ is a kernel function
* Without loss of generality, we assume that means are zeros


In [ ]:
# Finite number of points
X = np.arange(-5, 5, 0.2).reshape(-1, 1)

# Mean and covariance of the prior
mu = np.zeros(X.shape)
cov = kernel(X, X)

# Draw three samples from the prior
samples = np.random.multivariate_normal(mu.ravel(), cov, 3)
# Each sample is a guess of what function looks like!
# Since we haven't observed anything, all of them are equally likely.

# Plot GP mean, confidence interval and samples 
plot_gp(mu, cov, X, samples=samples)

## Prior probability distribution
* **A GP prior $p(f|X)$ can be converted to a GP posterior $p(f|X,y)$ after observing some data y.**

## Prior to Posterior: incorporating the observations

* By definition of GP, 
$$P(f|X) = N(\mu,\Sigma)$$ where $f = [f(x_1),...f(x_n)]$ and $X = [x_1,...,x_n]$



* If we observe values of function *f* for some inputs *x*, we now denote the following

    * $X$: Input points at which we made the observations
    * $y$: Evaluation of function *f* at $X$ $\rightarrow$ these are our observations!
    * $X_*$: Points at which we want to know or predict the values
    * $f_*$: The predicted output of the function at $X_*$

* Thus, we can write,
$$P(y,f_*|X,X_*) \rightarrow N(0,[[K_y,K_*],[K^T_*,K_{**}]])$$

<br>
* Here, $K_y=\kappa(X,X)$, $K_*=\kappa(X,X_*)$ and $K_{**}=\kappa(X_*,X_*)$


* Using Marginalization and some 4 pages of maths....
    $$p(f_*|X_*,X,y) =N(f_*|\mu_*,\Sigma_*)$$

* Where,
$$\mu_*=K_*^TK^{-1}_y y$$
$$\Sigma_* = K_{**} - K^T_* K^{-1}_y K_*$$

* An important point to observe: $y_i$ only appears in update of mean vector $\mu$. It doesn't influence the uncertainty $\Sigma$

* Refer to [this](http://www.gaussianprocess.org/gpml/) for a detailed proof of the above relations

In [ ]:
from numpy.linalg import inv

def posterior_predictive(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    '''  
    Computes the suffifient statistics of the GP posterior predictive distribution 
    from m training data X_train and Y_train and n new inputs X_s.
    
    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.
    
    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    '''
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)
    
    # Equation (4)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)

    # Equation (5)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
    
    return mu_s, cov_s

## Training a Gaussian Process:
* Training is equivalent to finding the posterior distribution given observed data (X) and inputs(y)

<br>
$$p(f_*|X_*,X,y) =N(f_*|\mu_*,\Sigma_*)$$

In [ ]:
# Noise free training data
X_train = np.array([-4, -3, -2, -1, 1]).reshape(-1, 1)
Y_train = np.sin(X_train)

# Define the testing space we want to probe
X = np.arange(-5, 5, 0.2).reshape(-1, 1)

# Compute mean and covariance of the posterior predictive distribution
mu_s, cov_s = posterior_predictive(X, X_train, Y_train)

samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, 3)
# get rid of the different samples
plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train, samples=samples)

## Consequences of GP
* Gaussian Process Regression gives us a way to quantify uncertainty 
* We use the covariance matrix to find the standard deviation
    * High deviation -> High uncertainity
* How is this useful?
    * We exploit the tradeoff between uncertainty and certainty
    * This exploitation is the basis of Bayesian Optimization!

## Bayesian Optimization basics

* Making smart setting choices 
![](pics_for_a3md/flow_bo.png)

* Based on initial or procured experimental datapoints, we use Bayesian Optimization to find new set of parameters which can help us direct our next set of experiments

* These points can be based either on the maximum uncertainity or maximum potential reward: this is typically referred to as a trade off between exploitation and exploration. 

## Bayesian Optimization

* Sequential design strategy for global optimization of black-box functions
* Usually employed to optimize expensive-to-evaluate functions
* Using information / data available to us to navigate the available exploration space
* This is very useful for experimentalists where performing every experiment is time consuming

### Let's define a few terms!
* Bayesian Optimization builds a probability model of the **objective function** to find the global optimimum in a minimum number of steps
* The **objective function** is the black-box-function we want to optimize $\rightarrow$ mapping experimental conditions to experimental outputs. (We will discuss this further)
* Bayesian optimization incorporates a prior belief about the objective function *f* and updates the prior with samples drawn from *f* to get a posterior that better approximates *f*.

![](pics_for_a3md/gp_model.png)

## Surrogate models & Uncertainty:
* Given the input data, we want to find the points of the input space which we expect to be highly rewarding. 
* This can be done with using uncertainities:
    * **Exploration**: Exploring regions of inputs where we have higher uncertainty
    * **Exploitation**: Choosing points in regions where we are more certain to get guarranted rewards
* Mathematically, uncertainity is quantified using probability. Gaussian Process (GP) helps us model uncertainties 
* Such a model is called a **Surrogate Model**

* **Acqusition function** is the way we specify the trade-off between exploration and exploitation

![](pics_for_a3md/time_evolution_gp_c.png)
* The blue region indicates the uncertainty of the output space.

### Acquisition function:

* Bayesian optimization uses an acquisition function that directs sampling to areas where an improvement over the current best observation is likely.
* Popular acquisition functions: maximum probability of improvement (MPI), expected improvement (EI) 

$$MPI(x) = \psi(\frac{\mu(x)-f(x^+)-\xi}{\sigma(x)})$$

$$EI(x) = (\mu(x)-f(x^+)-\xi) \psi(\frac{\mu(x)-f(x^+)-\xi}{\sigma(x)}) + \sigma(x) \phi(\frac{\mu(x)-f(x^+)-\xi}{\sigma(x)})$$

$$UCB(x) = \mu(x) + \beta \sigma(x)$$

where $\mu(x)$ is the mean function, $\sigma(x)$ is the variance, $\beta$ & $\xi$ are parameters controlling degree of exploration and $\psi(z)$ \& $\phi(z)$ is cumulative distribution function of a standard Gaussian distribution $N(0,1)$


### The Bayesian Optimization Framework:

* The Bayesian optimization procedure is as follows. For t=1,2,… repeat:
    * Find the next sampling point $x_t$ by optimizing the acquisition function over the GP: $x_t=argmax_xu(x|D_{1:t−1})$
    * Obtain a possibly noisy sample $y_t=f(x_t)+ϵ_t$ from the objective function f.
    * Add the sample to previous samples $D_{1:t}=\{D_{1:t−1},(x_t,y_t)\}$ and update the GP.

## Bayesian Optimization in Python

* Several libraries are available for performing bayesian optimization in Python
* We will use two libraries: scikit-learn and bayesian-optimization
* We will use scikit-learn for developing surrogate model; and the bayesian-optimization library  for the optimization
* Lets install the required packages:

In [ ]:
!pip install bayesian-optimization
!pip install scikit-learn

### Let's solve a dummy problem with Bayesian Optimization 

* We will choose a dummy function which we will probe
* Using Bayesian Optimization, we will try to find the point where it achieves the maximum
* black_box_function can return a truth table as well
    * This is what you will use for experimental data!

In [ ]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.
    In our case, this will be a simple ML model trained on experimental datapoints
    """
    return -x ** 2 - (y - 1) ** 2 + 1

### Let's introduce the BayesianOptimization object

In [ ]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

* f is the function to be optimized and expensive to compute
* pbounds: bounds on the parameters

### Let's perform the maximize operation 

In [ ]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

* init_points: Represent number of steps of random experiments we want to perform
* n_iter: Steps of Bayesian Optimization we want to perform

### Let's look at the point where we are expect to perform our next experiment

In [ ]:
print(optimizer.max)

### What if we don't want to train an ML model ?
* We can specify the points we want the framework to probe

* Our black_box_function can then provide the experimental data we already know

In [ ]:
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)

# Will probe only the two points specified above
optimizer.maximize(init_points=0, n_iter=0)

### Including several pre-defined points from experiments

In [ ]:
vals = [[0.5,0.6],[0.9,0.4],[0.3,0.88]]

for i in vals:
    optimizer.probe( params=i,lazy=True,)


# Will probe only the two points specified above
optimizer.maximize(init_points=0, n_iter=0)

### Saving the trained models for later usage

* Will save things in a .json format

In [ ]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

# Results will be saved in ./logs.json
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

### Loading the progress

In [ ]:
from bayes_opt.util import load_logs


new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)

# New optimizer is loaded with previously seen points
load_logs(new_optimizer, logs=["./logs.json"]);

### How do we deal with discrete variables?

* Till now all the exercises were with continuous variables
* There is no direct way to handle discrete variables but we can do some tricks

In [ ]:
def function_to_be_optimized(x, y, w):
    d = int(w)
    return ((x + y + d) // (1 + d)) / (1 + (x + y) ** 2)

In [ ]:
optimizer = BayesianOptimization(f=function_to_be_optimized, pbounds={'x': (-10, 10), 'y': (-10, 10), 'w': (0, 5)})
optimizer.maximize(init_points=2,n_iter=2)

### Can we modify the hyper-parameters of the underlying GP?
* The answer is yes; it can be done while calling the maximize routine
* The backend uses [sklearn's Gaussian Process Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html)
* So, any parameter that is supported there can be given as an input

In [ ]:
optimizer.maximize(
    init_points=1,
    n_iter=2,
    # What follows are GP regressor parameters
    alpha=1e-3,
    n_restarts_optimizer=5
)

# Thank you!

## Appendix:
* Bayesian Neural Networks (BNNs) 

### Bayesian Neural Networks (BNNs)

* BNN is a neural network with a prior distribution on its weights
* The known data is used to update the posterior given our prior beliefs about the weights
* Standard NN training via optimization is equivalent to maximum likelihood estimation (MLE) for the weights

## Maths of BNNs:
* We are given data set $\{(x_n,y_n)\}$, where each data point comprises of features $x_n \in R^D$ and output $y_n \in R$. 
* Define the likelihood for each data point as,
$$p(y_n|w,x_n,\sigma^2) = N(y_n|h(x_n;w),\sigma^2)$$
where $h(\cdot)$ denotes a neural network whose weights and biases form the latent variables **w**. Assume $\sigma^2$ is known. 
* We also define the prior on the weights and biases w to be standard normal:
$$p(w) = N(0,1)$$
* A very easy way to run a BNN is through [Edward library](http://edwardlib.org/tutorials/bayesian-neural-network)